# BE - Machine Learning

### Données

Le dataset est un dataset UCML, on pourrait utiliser le module *ucmlrepo* et la methode *fetch_ucirepo* associée, mais le chargement des données est long (~2 minutes). On privilégiera donc un téléchargement et des méthodes de lecture manuelles, les données ne faisant que 600Ko

Le dataset a été construit pour prédire la target *income* qui comporte deux valeurs (>50K, <=50K) à partir de 14 features. Cette construction en "classes" nous impose de travailler avec des classifiers plutôt qu'avec des regressions.

In [50]:
import pandas as pd

names = pd.read_table("data/adult.names", sep=":", skiprows=96, header=None)[0].tolist()
names.append("income") # La target n'est pas dans le fichier des noms
adult = pd.read_csv("data/adult.data", sep=", ", names=names, engine='python', index_col=False) # Le engine permet juste d'éviter un warning

### Nettoyage et Pre processing

Avant de mettre en place des modèles il nous fait nettoyer un peu les données pour pouvoir les exploiter. En particulier, nous aimerions supprimer les *valuers manquantes* ainsi que les *outliers*. En mettant en place les algo classiques, nous nous sommes vite rendu compte d'un problème : la plupart des catégories sont encodées sous forme de texte. Ainsi les algo ne peuvent fonctionner. C'est la raison pour laquelle nous allons mettre en place cette pipeline de nettoyage :
- **Suppression des #NA :** Les NA étant encodés par des "?" (il n'y a pas de NA sinon) nous allons supprimer chaque lignes qui en comportent
- **Encodage des colonnes :** Nous allons encoder chaque colonnes de la façon suivante : chaque valeur unique se voit attribuer un entier. Nous n'avons que peu de recul pour l'instant et ne savons pas si c'est pertinent
- **Normalisation :** POur contrer l'effet du nombre e colonnes, nous allons normaliser celles ci


In [ ]:
#### Suppression des NA ####
adult.replace("?", pd.NA, inplace=True)  # Les NA sont codés par des "?"

for key in adult.keys():
    n_na = adult[key].isna().sum()
    if n_na > 0:
        print(f"{key} : {n_na} NA")

adult.dropna(inplace=True)  # On supprime les lignes avec NA

workclass : 1836 NA
occupation : 1843 NA
native-country : 583 NA


In [ ]:
#### Encodage des colonnes ####
from sklearn.preprocessing import LabelEncoder

label_encoders = {}
for column in adult.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    adult[column] = le.fit_transform(adult[column])
    label_encoders[column] = le